In [1]:
import sqlite3
import pandas as pd

In [15]:
## Load Visualization Pipeline for December 2023
#!python3 -m scripts.visualization_pipeline
#!python3 -m scripts.mappings_host_location

In [16]:
import sqlite3
import pandas as pd

def create_connection(db_file):
    conn = sqlite3.connect(db_file)
    return conn

def create_table(conn):
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS my_table (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            column1 TEXT,
            column2 INTEGER,
            column3 REAL
        )
    ''')
    conn.commit()

def insert_dataframe(conn, df):
    df.to_sql('listings', conn, if_exists='replace', index=False)
    
def main():
    database = 'data/database.sqlite'
    conn = create_connection(database)
    
    create_table(conn)
    df = pd.read_pickle("data/pickles/december_listings_viz.pkl")
    insert_dataframe(conn, df)
    
    conn.close()
    
main()

In [2]:
def load_listings_datasets():
    datasets = {'2023dic': pd.read_csv("data/2023dic/d_listings.csv"),
                '2023sep': pd.read_csv("data/2023sep/d_listings.csv"),
                '2024jun': pd.read_csv("data/2024jun/d_listings.csv"),
                '2024mar': pd.read_csv("data/2024mar/d_listings.csv")}
    return datasets

In [3]:
dataframes = load_listings_datasets()

In [35]:
dic = dataframes['2024jun']